In [0]:
# Requirements: pymongo, mongod, (optional: mongodb GUI)
import pymongo
from pprint import pprint

client = pymongo.MongoClient('localhost', 27017)
client.drop_database('test')
db = client.test

In [0]:
reviews = [
    {
        'product_id': 0,
        'name': 'semin',
        'age': 24,
        'text': '...'
    },
    {
        'product_id': 0,
        'name': 'andy',
        'age': 24,
        'text': '...'
    },
    {
        'product_id': 0,
        'name': 'eli',
        'age': 25,
        'text': '...'
    },
    {
        'product_id': 1,
        'name': 'semin',
        'age': 24,
        'text': '...'
    },
    {
        'product_id': 1,
        'name': 'tom',
        'age': 17,
        'text': '...'
    },
]

db.reviews.insert_many(reviews)

### Operators

- `$project`: field 필터링 (name만 보여주고 싶다면 `{$project: {'name': 1}}` 와 같은 식으로 작성)
- `$match`: document 필터링 (제시하는 조건에 부합하는 document만 다음 파이프라인으로 넘어가게 된다.
- `$limit`: 다음 파이프라인에 넘겨줄 도큐멘트의 수를 제한한다.
- `$skip`: n개의 도큐멘트를 건너뛴다.
- `$group`: groupby
- `$sort`: 결과를 정렬해준다
- `$out`: 결과를 다른 collection에 넣어준다. 만약 collection이 아직 생성되지 않았다면 새로 생성한 후 결과 도큐멘트들을 넣어주고, 이미 존재하는 경우 덮어쓴다.

In [0]:
# 각 product마다 몇개의 리뷰가 존재하는가
out = db.reviews.aggregate([
    {
        '$group': {
            '_id': '$product_id',
            'count': {'$sum': 1}
        }
    }
])
pprint(list(out))

[{'_id': 0, 'count': 3}, {'_id': 1, 'count': 2}]


In [0]:
# 특정 product만 셀거라면 아래의 방법도 가능하다.
out = db.reviews.count_documents({'product_id': 0})
pprint(out)

3


In [0]:
# 0번 product에 대해 나이별 리뷰 수는 어떻게 되는지를 summary 컬렉션에 저장
out = db.reviews.aggregate([
    {
        '$match': {
            'product_id': 0
        }
    },
    {
        '$group': {
            '_id': '$age',
            'cnt': {'$sum': 1},
        }
    },
    {
        '$out': 'summary'
    }
])
pprint(list(out))

[]


In [0]:
# DB 확인
db.summary.find_one()

{'_id': 25, 'cnt': 1}

In [0]:
# 리뷰한 사람의 이름만 다음 aggregation step에 넘겨줌
out = db.reviews.aggregate([
    {
        '$project': {'name': 1}
    },
])
pprint(list(out))

[{'_id': ObjectId('5dbc8e14c0607cf3256da7bb'), 'name': 'semin'},
 {'_id': ObjectId('5dbc8e14c0607cf3256da7bc'), 'name': 'andy'},
 {'_id': ObjectId('5dbc8e14c0607cf3256da7bd'), 'name': 'eli'},
 {'_id': ObjectId('5dbc8e14c0607cf3256da7be'), 'name': 'semin'},
 {'_id': ObjectId('5dbc8e14c0607cf3256da7bf'), 'name': 'tom'}]


In [0]:
# 리뷰작성자 3명만 확인
out = db.reviews.aggregate([
    {
        '$project': {'name': 1}
    },
    {
        '$limit': 3
    }
])
pprint(list(out))

[{'_id': ObjectId('5dbc8e14c0607cf3256da7bb'), 'name': 'semin'},
 {'_id': ObjectId('5dbc8e14c0607cf3256da7bc'), 'name': 'andy'},
 {'_id': ObjectId('5dbc8e14c0607cf3256da7bd'), 'name': 'eli'}]


In [0]:
# 처음 두개의 리뷰는 무시하고, 나머지 리뷰의 작성자만 확인
out = db.reviews.aggregate([
    {
        '$project': {'name': 1}
    },
    {
        '$skip': 2
    }
])
pprint(list(out))

[{'_id': ObjectId('5dbc8e14c0607cf3256da7bd'), 'name': 'eli'},
 {'_id': ObjectId('5dbc8e14c0607cf3256da7be'), 'name': 'semin'},
 {'_id': ObjectId('5dbc8e14c0607cf3256da7bf'), 'name': 'tom'}]


In [0]:
# 제품마다 리뷰의 개수와 리뷰자의 평균 나이
out = db.reviews.aggregate([
    {
        '$group': {
            '_id': '$product_id',
            'count': {'$sum': 1},
            'avg_age': {'$avg': '$age'}
        }
    },
])
pprint(list(out))

[{'_id': 0, 'avg_age': 24.333333333333332, 'count': 3},
 {'_id': 1, 'avg_age': 20.5, 'count': 2}]


In [0]:
out = db.reviews.aggregate([
    {
        '$project': {
            'properties': {
                'name': '$name',
                'age': '$age'
            },
            'flat_properties': {
                # '$concat': ['$name', '_', '$age'] -- 이렇게 하면 age가 int라서 concat이 안된다.
                '$concat': ['$name', '_', {'$substr': ['$age', 0, -1]}]
            }
        }
    },
])
pprint(list(out))

[{'_id': ObjectId('5dbc8e14c0607cf3256da7bb'),
  'flat_properties': 'semin_24',
  'properties': {'age': 24, 'name': 'semin'}},
 {'_id': ObjectId('5dbc8e14c0607cf3256da7bc'),
  'flat_properties': 'andy_24',
  'properties': {'age': 24, 'name': 'andy'}},
 {'_id': ObjectId('5dbc8e14c0607cf3256da7bd'),
  'flat_properties': 'eli_25',
  'properties': {'age': 25, 'name': 'eli'}},
 {'_id': ObjectId('5dbc8e14c0607cf3256da7be'),
  'flat_properties': 'semin_24',
  'properties': {'age': 24, 'name': 'semin'}},
 {'_id': ObjectId('5dbc8e14c0607cf3256da7bf'),
  'flat_properties': 'tom_17',
  'properties': {'age': 17, 'name': 'tom'}}]


### unwind

`$unwind` 예제: https://stackoverflow.com/questions/16448175/whats-the-unwind-operator-in-mongodb